<a href="https://colab.research.google.com/github/0xdferraz/Saense-PLN/blob/main/abstractive_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**0xdferraz/Saense-PLN ( https://github.com/0xdferraz/Saense-PLN )**

"mT5_multilingual_XLSum" por csebuetnlp, disponível em: https://huggingface.co/csebuetnlp/mT5_multilingual_XLSum

**Intruções:**

*   Faça upload da pasta contendo os artigos extraídos com o algoritmo "handle_site.py"
*   Crie uma pasta chamada "resumos"
*   Execute o algoritmo
*   Opcional: Ao final você pode criar uma pasta chamada "zip" e executar a seguinte linha: ``` !zip -r zip/resumos.zip resumos/ ```. Assim, será mais fácil baixar os resultados.







In [ ]:
!pip install sentencepiece

In [ ]:
!pip install transformers

In [ ]:
import transformers
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import glob
import os

In [ ]:
WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

In [ ]:
model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
for file in glob.glob("artigos/*.txt"):
  if file != "artigos/__controle.txt":
    f = open(file,'r',encoding = 'utf-8')
    article_text = f.read()
    article_name = str(file).split('/')[-1]
    length = len(article_text)

    input_ids = tokenizer(
        [WHITESPACE_HANDLER(article_text)],
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=512
    )["input_ids"]

    output_ids = model.generate(
        input_ids=input_ids,
        max_length=84,
        no_repeat_ngram_size=2,
        num_beams=4
    )[0]

    summary = tokenizer.decode(
        output_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    )


    nomearquivo = "Summary_" + article_name
    nomecompleto = os.path.join("./resumos", nomearquivo)
    f = open(nomecompleto, "w")
    f.write(summary)
    f.close()


In [ ]:
## Opcinal ##
!zip -r zip/resumos.zip resumos/